<a href="https://colab.research.google.com/github/mamba124/just_for_fun/blob/master/Denoising_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras import utils 
from tensorflow.keras.preprocessing import image 
import numpy as np 
import os 
import pandas as pd 
from tensorflow.keras.models import Model, load_model

In [0]:
st = '/content/drive/My Drive/train/clean/47/47_122796_47-122796-0071 (1).npy'
stop = st[-7:]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def get_chunks(signal, length, step):
  batch =[]
  syms_len = len(signal)
  index = 0
  
  while (index + length <= syms_len):
    batch.append(signal[index:index+length])
    index += step

  return batch

def prepare_chunks(audio,length, step):
  batch_list = get_chunks(audio, length, step)
  return np.array(batch_list)

In [0]:
directory = '/content/drive/My Drive/train/'
folders = os.listdir(directory)
clean_arrays = []
noisy_arrays = []
patch = os.listdir(directory+'clean/')

for folder in patch:
  for arr in os.listdir(directory+'clean/'+folder):
    if arr[-7:]!=stop:
      clean_arrays.append(np.load(directory+'clean/'+folder+'/'+arr))
      noisy_arrays.append(np.load(directory+'noisy/'+folder+'/'+arr))



In [0]:
clean_arrays_ = []
noisy_arrays_ = []

for i in range(len(clean_arrays[:-1])):
  for j in range(len(clean_arrays[i])):
    clean_arrays_.append(clean_arrays[i][j])
    noisy_arrays_.append(noisy_arrays[i][j])

In [0]:
length = 60 
step = 30

x_trainC = prepare_chunks(clean_arrays_, length, step)
x_trainN = prepare_chunks(noisy_arrays_, length, step)

y_trainC = np.ones((x_trainC.shape[0],1))
y_trainN = np.zeros((x_trainN.shape[0],1))

In [0]:
class paired:
  def __init__(self, cleared, noised):
    self.clear = cleared
    self.noise = noised

  def __getitem__(self,index):
    return self.clear[index], self.noise[index]

  def __len__(self):
    return len(self.clear)


In [0]:
def shuffle(a, b):
    p = np.random.permutation(len(b)) 
    permuted_ = a[p], b[p]
    X_input = []
    Y_input = []
    permuted_output = []
    choice = np.random.choice([0,1],size = len(permuted_[0][0]), p = [0.5,0.5])
    for att in range(len(permuted_[0][0])):
      X_input.append(permuted_[0][choice[att]][att])
      Y_input.append(permuted_[1][choice[att]][att])
      permuted_output.append(permuted_[0][0][att])
    return (X_input,Y_input), permuted_output
    

    return 
#x_train = np.concatenate([X, X], axis = 0)
#y_train = np.concatenate([y_trainC, y_trainN], axis = 0)
X = paired(x_trainC, x_trainN)
Y = paired(y_trainC, y_trainN)

#x_val = np.concatenate([x_valC, x_valN], axis = 0)
#y_val = np.concatenate([y_valC, y_valN], axis = 0)

(x_train_, y_train_), output = shuffle(X, Y)
x_train = np.array(x_train_)
y_train = np.array(y_train_)
output = np.array(output)
#x_val_, y_val_ = shuffle(x_val, y_val)

In [0]:
def hybrid(vector):

  inputs_vector = layers.Input(shape = (vector.shape[1],vector.shape[2]))

  ##Classifier##
  out_class = layers.BatchNormalization()(inputs_vector)
  out_class= layers.Conv1D(64, 3,strides = 2, padding = 'same')(out_class)
  out_class = layers.Flatten()(out_class)
  out_class = layers.Dense(200, activation='elu')(out_class)
  out_class = layers.Dense(100, activation='elu')(out_class)
  out_class = layers.Dense(56, activation = 'relu')(out_class)
  out_class = layers.Dense(1, activation = 'sigmoid')(out_class)
  
  ###Encoder###
  ##Block1##
  hidden = layers.Conv1D(4, 3, padding = 'same') (inputs_vector)
  hidden = layers.Activation('tanh')(hidden)
 # hidden = layers.BatchNormalization()(hidden)

  hidden = layers.Conv1D(4, 3, padding = 'same')(hidden)
  hidden = layers.Activation('relu')(hidden)
 # hidden = layers.BatchNormalization()(hidden)

  hidden = layers.MaxPooling1D(3)(hidden)

  ##Block2##
  hidden = layers.Conv1D(32, 3, padding = 'same') (hidden)
  #hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(32, 3, padding = 'same', strides = 2)(hidden)
  hidden = layers.Activation('relu')(hidden)
 # hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(3)(hidden)
  
  ##Block3##
  hidden = layers.Conv1D(128, 3, padding = 'same') (hidden)
  #hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(3)(hidden)  
  hidden = layers.Multiply()([1-out_class, hidden])
  
  ###Decoder###
  ##UPBlock1##
  hidden = layers.UpSampling1D(3)(hidden)
  hidden = layers.Conv1D(128, 3, padding = 'same', activation= 'relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)

  ##UPBlock2##
  hidden = layers.UpSampling1D(4)(hidden)
  hidden = layers.Conv1D(32, 3, padding = 'same', activation= 'tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(32, 3, padding = 'same', activation= 'tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)

  ##UPblock3##
  hidden = layers.UpSampling1D(5)(hidden)
  hidden = layers.Conv1D(4, 3, padding = 'same',  activation= 'tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(4, 3, padding = 'same',  activation= 'tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)

  out = layers.Conv1D(80, 3, padding = 'same',  activation= 'tanh')(hidden)
 # hidden = layers.Flatten()(hidden)
 # hidden = layers.Dense(156, activation='relu')(hidden)
 # hidden = layers.Dense(256, activation='elu')(hidden)
 # out = layers.Dense(60*80, activation = 'tanh')(hidden)
 # out = layers.Reshape((60,80))(out)
  modelD = Model(inputs=inputs_vector, outputs = [out, out_class])
  #modelD.summary()

  return modelD


  

In [101]:
denoise = hybrid(x_train)
denoise.compile(optimizer = optimizers.RMSprop(), loss = ['mse','binary_crossentropy'], metrics = ['mse','accuracy'])
denoise.fit(x_train, [output, y_train], validation_split = 0.3, batch_size=500, epochs = 20)

Epoch 1/20
136/136 [==============================] - 14s 103ms/step - loss: 0.5075 - conv1d_114_loss: 0.1868 - dense_54_loss: 0.3207 - conv1d_114_mse: 0.1870 - conv1d_114_accuracy: 0.0562 - dense_54_mse: 0.0955 - dense_54_accuracy: 0.8693 - val_loss: 0.7015 - val_conv1d_114_loss: 0.1557 - val_dense_54_loss: 0.5459 - val_conv1d_114_mse: 0.1556 - val_conv1d_114_accuracy: 0.0559 - val_dense_54_mse: 0.1749 - val_dense_54_accuracy: 0.7548
Epoch 2/20
136/136 [==============================] - 12s 91ms/step - loss: 0.3195 - conv1d_114_loss: 0.1237 - dense_54_loss: 0.1958 - conv1d_114_mse: 0.1237 - conv1d_114_accuracy: 0.0919 - dense_54_mse: 0.0567 - dense_54_accuracy: 0.9239 - val_loss: 0.3542 - val_conv1d_114_loss: 0.1795 - val_dense_54_loss: 0.1747 - val_conv1d_114_mse: 0.1792 - val_conv1d_114_accuracy: 0.0841 - val_dense_54_mse: 0.0495 - val_dense_54_accuracy: 0.9338
Epoch 3/20
136/136 [==============================] - 12s 92ms/step - loss: 0.2671 - conv1d_114_loss: 0.1124 - dense_54_los

In [105]:
denoise.compile(optimizer = optimizers.RMSprop(0.0001), loss = ['mse','binary_crossentropy'], metrics = ['mse','accuracy'])
denoise.fit(x_train, [output, y_train], validation_split = 0.3, batch_size=500, epochs = 5)

Epoch 1/5
136/136 [==============================] - 14s 105ms/step - loss: 0.0927 - conv1d_114_loss: 0.0874 - dense_54_loss: 0.0053 - conv1d_114_mse: 0.0874 - conv1d_114_accuracy: 0.1089 - dense_54_mse: 8.9393e-04 - dense_54_accuracy: 0.9989 - val_loss: 0.3215 - val_conv1d_114_loss: 0.0892 - val_dense_54_loss: 0.2323 - val_conv1d_114_mse: 0.0892 - val_conv1d_114_accuracy: 0.1068 - val_dense_54_mse: 0.0466 - val_dense_54_accuracy: 0.9454
Epoch 2/5
136/136 [==============================] - 14s 100ms/step - loss: 0.0900 - conv1d_114_loss: 0.0868 - dense_54_loss: 0.0031 - conv1d_114_mse: 0.0869 - conv1d_114_accuracy: 0.1079 - dense_54_mse: 3.5693e-04 - dense_54_accuracy: 0.9996 - val_loss: 0.3242 - val_conv1d_114_loss: 0.0886 - val_dense_54_loss: 0.2357 - val_conv1d_114_mse: 0.0886 - val_conv1d_114_accuracy: 0.1071 - val_dense_54_mse: 0.0464 - val_dense_54_accuracy: 0.9459
Epoch 3/5
136/136 [==============================] - 14s 99ms/step - loss: 0.0888 - conv1d_114_loss: 0.0863 - dense_

In [103]:
denoise.predict(np.expand_dims(x_train[-2],0))

[array([[[ 0.3973342 ,  0.5374595 ,  0.64984053, ..., -0.02274307,
          -0.04630992, -0.06243704],
         [ 0.3723632 ,  0.5535041 ,  0.6937918 , ...,  0.00326012,
          -0.01283165, -0.02297579],
         [ 0.4338003 ,  0.61463916,  0.7306186 , ...,  0.04649654,
           0.03938435,  0.02688539],
         ...,
         [ 0.36309382,  0.52069145,  0.62091804, ..., -0.10670038,
          -0.11863761, -0.12855522],
         [ 0.3098724 ,  0.4789769 ,  0.58413786, ..., -0.1266183 ,
          -0.1363476 , -0.15849754],
         [ 0.3598903 ,  0.44390488,  0.568108  , ..., -0.1083078 ,
          -0.15977377, -0.15403005]]], dtype=float32),
 array([[5.3979288e-09]], dtype=float32)]

In [104]:
output[-2]

array([[ 0.2496 ,  0.4897 ,  0.557  , ...,  0.3127 ,  0.1647 ,  0.2498 ],
       [ 0.278  ,  0.5083 ,  0.5776 , ...,  0.3447 ,  0.2256 ,  0.312  ],
       [ 0.1466 ,  0.5034 ,  0.596  , ...,  0.2468 ,  0.186  ,  0.2588 ],
       ...,
       [ 0.2273 ,  0.5527 ,  0.6304 , ..., -0.1467 , -0.1864 , -0.3389 ],
       [ 0.284  ,  0.5146 ,  0.62   , ..., -0.1223 , -0.1646 , -0.294  ],
       [ 0.3284 ,  0.482  ,  0.606  , ...,  0.01232, -0.09705, -0.1605 ]],
      dtype=float16)

In [0]:
directory_dataset = '/content/drive/My Drive/train/noisy/1006'#input('dataset:')
directory_result = './'+'results/'#directory_dataset+'/'
os.mkdir(directory_result + 'denoised/')

result = []
for curr_file in os.listdir(directory_dataset):

  arr = np.load(directory_dataset + '/' + curr_file)
  test = prepare_chunks(arr, length, step)
  denoised, classes = denoise.predict(test)
  curr_class = round(classes.sum()/classes.shape[0])
  result.append([curr_file, curr_class])

  if curr_class==0:
    signals = []
    for signal in np.expand_dims(denoised,0):
      glued = []
      for i in range(len(signal)):
        if i == 0: ind = 0
        else: ind = length - step
        for j in range(ind,length):
          glued.append(signal[i][j])
      signals.append(np.array(glued))
    np.save(directory_result + 'denoised/' + curr_file, np.array(signals))

result = np.array(result)
result = pd.DataFrame({'id':result[:,0], 'class':result[:,1]})
result.to_csv(directory_result + '/result.csv')